# Comebine the data into a single database

In [50]:
import numpy as np
import pandas as pd
import sqlite3

In [51]:
def load_tweet_data(df, db_path, append=True):
    
    conn = sqlite3.connect(db_path)
    if append:
        df.to_sql('tweets', con = conn, if_exists='append', index=False)
    else:
        df.to_sql('tweets', con = conn, if_exists='replace', index=False)
    conn.commit()
    conn.close()
    
    return None

In [90]:
# Execute this code cell to run the ETL pipeline


# Extract

PATHS_TO_DATA = ['data/disaster_response_messages_training.csv',
                 'data/disaster_response_messages_test.csv',
                 'data/disaster_response_messages_validation.csv'
                ]

df = pd.DataFrame()
for path in PATHS_TO_DATA:
    df_part = pd.read_csv(path)
    if df.empty:
        df = df_part
    else:
        df = pd.merge(df, df_part, how='outer')
    
# Transform

# Drop columns where every entry is the same
for col in df:
    if df[col].value_counts().shape[0] == 1:
        df.drop(columns=col, inplace=True)

# Drop the split column
df.drop(columns='split', inplace=True)
        
# The related column has a few tweets that are labeled as a 2 instead of a 1 or zero, drop these.
df = df[df['related'] != 2]

# Load

DB_PATH = 'data/disaster_tweets.db'
load_tweet_data(df, DB_PATH, append=False)

In [91]:
conn = sqlite3.connect(db_path)

pd.read_sql('SELECT * FROM tweets', con = conn).shape

(26055, 39)